## Better retrieval via "Dense Passage Retrieval"


### Importance of Retrievers

The Retriever has a huge impact on the performance of our overall search pipeline.


### Different types of Retrievers
#### Sparse
Family of algorithms based on counting the occurences of words (bag-of-words) resulting in very sparse vectors with length = vocab size. 

Examples: BM25, TF-IDF  
Pros: Simple, fast, well explainable  
Cons: Relies on exact keyword matches between query and text  
 

#### Dense
These retrievers use neural network models to create "dense" embedding vectors. Within this family there are two different approaches: 

a) Single encoder: Use a **single model** to embed both query and passage.  
b) Dual-encoder: Use **two models**, one to embed the query and one to embed the passage

Recent work suggests that dual encoders work better, likely because they can deal better with the different nature of query and passage (length, style, syntax ...). 

Examples: REALM, DPR, Sentence-Transformers ...
Pros: Captures semantinc similarity instead of "word matches" (e.g. synonyms, related topics ...) 
Cons: Computationally more heavy, initial training of model  


### "Dense Passage Retrieval"

In this Tutorial, we want to highlight one "Dense Dual-Encoder" called Dense Passage Retriever. 
It was introdoced by Karpukhin et al. (2020, https://arxiv.org/abs/2004.04906. 

Original Abstract: 

_"Open-domain question answering relies on efficient passage retrieval to select candidate contexts, where traditional sparse vector space models, such as TF-IDF or BM25, are the de facto method. In this work, we show that retrieval can be practically implemented using dense representations alone, where embeddings are learned from a small number of questions and passages by a simple dual-encoder framework. When evaluated on a wide range of open-domain QA datasets, our dense retriever outperforms a strong Lucene-BM25 system largely by 9%-19% absolute in terms of top-20 passage retrieval accuracy, and helps our end-to-end QA system establish new state-of-the-art on multiple open-domain QA benchmarks."_

Paper: https://arxiv.org/abs/2004.04906  
Original Code: https://fburl.com/qa-dpr 


*Use this [link](https://colab.research.google.com/github/deepset-ai/haystack/blob/master/tutorials/Tutorial6_Better_Retrieval_via_DPR.ipynb) to open the notebook in Google Colab.*


## Prepare environment

### Colab: Enable the GPU runtime 
Make sure you enable the GPU runtime to experience decent speed in this tutorial.  
**Runtime -> Change Runtime type -> Hardware accelerator -> GPU**

<img src="https://raw.githubusercontent.com/deepset-ai/haystack/master/docs/img/colab_gpu_runtime.jpg">

In [ ]:
# Make sure you have a GPU running
!nvidia-smi

In [ ]:
! pip install git+https://github.com/deepset-ai/haystack.git

In [7]:
from haystack import Finder
from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

## Document Store

### Start an Elasticsearch server
You can start Elasticsearch on your local machine instance using Docker. If Docker is not readily available in your environment (e.g. in Colab notebooks), you can also manually download and execute Elasticsearch from source.

In [8]:
# Recommended: Start Elasticsearch using Docker
#! docker run -d -p 9200:9200 -e "discovery.type=single-node" elasticsearch:7.6.2
# wait until ES has started
#! sleep 30

4cc342fd2f0096e6335390e66029716ef452e2853ddd11a5b9802a4fdde20cdc


In [9]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [12]:
# Connect to Elasticsearch
from haystack.database.elasticsearch import ElasticsearchDocumentStore

# We need to set `embedding_field` and `embedding_dim`, when we plan to use a dense retriever
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document", 
                                            embedding_field="embedding", embedding_dim=768)

07/03/2020 11:46:26 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.343s]


## Cleaning & indexing documents

Similarly to the previous tutorials, we download, convert and index some Game of Thrones articles to our DocumentStore

In [13]:
# Let's first get some files that we want to use
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Convert files to dicts
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(dicts)

07/03/2020 11:46:28 - INFO - haystack.indexing.utils -   Found data stored in `data/article_txt_got`. Delete this first if you really want to fetch new data.
07/03/2020 11:46:28 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.300s]
07/03/2020 11:46:28 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.209s]
07/03/2020 11:46:28 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.124s]
07/03/2020 11:46:29 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.101s]
07/03/2020 11:46:29 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.125s]
07/03/2020 11:46:29 - INFO - elasticsearch -   POST http://localhost:9200/_bulk [status:200 request:0.096s]


## Initalize Retriever, Reader,  & Finder

### Retriever

**Here:** We use a `DensePassageRetriever`

**Alternatives:**

- The `ElasticsearchRetriever`with custom queries (e.g. boosting) and filters
- Use `EmbeddingRetriever` to find candidate documents based on the similarity of embeddings (e.g. created via Sentence-BERT)
- Use `TfidfRetriever` in combination with a SQL or InMemory Document store for simple prototyping and debugging

In [14]:
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store, embedding_model="dpr-bert-base-nq",
                                  do_lower_case=True, gpu=True)

# Important: 
# Now that after we have the DPR initialized, we need to call update_embeddings() to iterate over all
# previously indexed documents and update their embedding representation. 
# While this can be a time consuming operation (depending on corpus size), it only needs to be done once. 
# At query time, we only need to embed the query and compare it the existing doc embeddings which is very fast.
document_store.update_embeddings(retriever)

07/03/2020 11:46:29 - INFO - haystack.retriever.dpr_utils -   Loading saved model from models/dpr/checkpoint/retriever/single/nq/bert-base-encoder.cp
07/03/2020 11:46:29 - INFO - haystack.retriever.dense -   Loaded encoder params:  {'do_lower_case': True, 'pretrained_model_cfg': 'bert-base-uncased', 'encoder_model_type': 'hf_bert', 'pretrained_file': None, 'projection_dim': 0, 'sequence_length': 256}
07/03/2020 11:46:36 - INFO - haystack.retriever.dense -   Loading saved model state ...
07/03/2020 11:46:36 - INFO - haystack.retriever.dense -   Loading saved model state ...
07/03/2020 11:46:37 - INFO - elasticsearch -   GET http://localhost:9200/document/_search?scroll=5m&size=1000 [status:200 request:0.177s]
07/03/2020 11:46:37 - INFO - elasticsearch -   GET http://localhost:9200/_search/scroll [status:200 request:0.053s]
07/03/2020 11:46:37 - INFO - elasticsearch -   GET http://localhost:9200/_search/scroll [status:200 request:0.047s]
07/03/2020 11:46:37 - INFO - elasticsearch -   GET

### Reader

Similar to previous Tutorials we now initalize our reader.

Here we use a FARMReader with the *deepset/roberta-base-squad2* model (see: https://huggingface.co/deepset/roberta-base-squad2)



#### FARMReader

In [15]:
# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

07/03/2020 12:00:52 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
07/03/2020 12:00:52 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
07/03/2020 12:00:59 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
07/03/2020 12:01:07 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"loss_ignore_index": -1}
/home/mp/miniconda3/envs/py37/lib/python3.7/site-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
07/03/2020 12:01:11 - INFO - farm.utils -   devic

### Finder

The Finder sticks together reader and retriever in a pipeline to answer our actual questions. 

In [16]:
finder = Finder(reader, retriever)

## Voilà! Ask a question!

In [25]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers. 
prediction = finder.get_answers(question="Who created the Dothraki vocabulary?", top_k_retriever=10, top_k_reader=5)

#prediction = finder.get_answers(question="Who is the father of Arya Stark?", top_k_retriever=10, top_k_reader=5)
#prediction = finder.get_answers(question="Who is the sister of Sansa?", top_k_retriever=10, top_k_reader=5)

07/03/2020 12:07:16 - INFO - elasticsearch -   GET http://localhost:9200/document/_search [status:200 request:0.018s]
07/03/2020 12:07:16 - INFO - haystack.finder -   Reader is looking for detailed answer in 11362 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.10 Batches/s]


In [27]:
print_answers(prediction, details="minimal")

[   {   'answer': 'David J. Peterson',
        'context': "ge for ''Game of Thrones''\n"
                   'The Dothraki vocabulary was created by David J. Peterson '
                   'well in advance of the adaptation. HBO hired the Language '
                   'Creation'},
    {   'answer': 'David J. Peterson',
        'context': '\n'
                   '===Creation===\n'
                   'David J. Peterson, creator of the spoken Valyrian '
                   "languages for ''Game of Thrones''\n"
                   'To create the Dothraki and Valyrian languages to b'},
    {   'answer': 'David J. Peterson',
        'context': 'orld. The language was developed for the TV series by the '
                   'linguist David J. Peterson, working off the Dothraki words '
                   "and phrases in Martin's novels.\n"
                   ','},
    {   'answer': 'Peterson',
        'context': "e does not exist in the fictional world of ''A Song of Ice "
                   "and F